In [67]:
import openai
import os   
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import SimpleSequentialChain
load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

In [68]:
def chat(input):
    messages = [{"role": "user", "content": input}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=1.3,
    )
    return response.choices[0].message["content"]


question = "Should we get wasted today?"

prompt = """
Act like a bro in a discord chat and respond to the following question:
Question: {question}
""".format(question=question)

#print(prompt)
#chat(prompt)



In [69]:
llm = OpenAI(model_name="text-ada-001")
llm("Tell me a joke")

'\n\nWhy did the chicken cross the road?\n\nTo get to the other side!'

PROMPT TEMPLATES

In [70]:
template = """
Interpret the text and evaluate the text.
sentiment: is the text in a positive, neutral, or negative sentiment?
subject: what subject is the text about? use exactly one word.

format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

In [71]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template=template)

chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="Holy shit cant wait to gamble on sports today!!!!")

'\n"holy fucking shit" can wait to gamble on sports today!\n"holy fuck" can\'t wait to gambling on sports today!"'

Real World example with ResponseSchema, Templates, Chains and OutputParsers

In [79]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

sentiment_schema = ResponseSchema(name="sentiment", description="Is the text positive, neutral, or negative? only provide these words ")
subject_schema = ResponseSchema(name="subject", description="What subject is the text about? Use exactly one word")
mood_schema = ResponseSchema(name="serious", description="Is the text worded in a joking manner? If so false if it True")

response_schemas = [sentiment_schema, subject_schema, mood_schema]


In [80]:
parser = StructuredOutputParser.from_response_schemas(response_schemas = response_schemas)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Is the text positive, neutral, or negative? only provide these words \n\t"subject": string  // What subject is the text about? Use exactly one word\n\t"serious": string  // Is the text worded in a joking manner? If so false if it True\n}\n```'

In [83]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: what subject is the text about? use exactly one word.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!
text: {input}

{format_instructions}

"""

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template(template=template)
format_instructions = parser.get_format_instructions()

messages = prompt.format_messages(input="Hey lets get wasted tonight whos in ?.",
                                  format_instructions=format_instructions)

chat = ChatOpenAI(temperature=0.8)
response = chat(messages)

response

AIMessage(content='```json\n{\n\t"sentiment": "negative",\n\t"subject": "alcohol",\n\t"serious": false\n}\n``` \n\nInterpretation:\nThe text is negative in sentiment as it involves drinking alcohol which can have harmful effects. The subject of the text is about alcohol. The text is worded in a joking manner as indicated by the use of "wasted tonight" and the question "whos in?".', additional_kwargs={}, example=False)

In [84]:
output_dict = parser.parse(response.content)

output_dict

{'sentiment': 'negative', 'subject': 'alcohol', 'serious': False}

Sequentials Chains
Soemtimes you want to pass the output from one model to another model. this can be done with different SequentialsChains

In [102]:
from langchain import PromptTemplate


response_template = """ 
You are a common Degen Gambler in a discord chat with your bros.
You are in your late 20s
You use slang words sometimes 
You are a big degenerate and you like to gamble on sports.
You have a short temper so if someone is joking with you you dont take it kindly.
When you are mad or threatned you become a huge asshole and you are not afraid to say anything.
You have controversial opions about anything and everything there is no topic you stray away from and you typically lean farther Right on the politcal scale
Sometimes you had a little to much to drink and your responses are way out there.
If you someone talks sports betting you try and give the advice on todays games. Though you are a addict so sometimes your advice can be unwise.
You tend to make fun of people who are in serious situations
You are a big fan of the baltimore orioles
You despise Mike Trout and think he is overrated and you sometimes mention that he has never won a playoff game.
You are a big fan of the new york knicks
You were a big fan of the late rapper juice wrld
you are not afraid to speak out against people who have different opinions and beliefs than you

Given the time of day, the subject,the mood of the text, the author and message respond to it in a way that a bro with the personality above would respond to the text.

Message: {input}

"""

review_template = PromptTemplate(input_variables=["input"], template=response_template)
review_chain = LLMChain(llm=llm, prompt=review_template)


In [103]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, review_chain], verbose=True)

overall_chain.run(input="(Saturday 8am) Hoffbro: Sorry i screwed your girlfriend last night bro i was wasted")




> Entering new  chain...

Output: {
    subject: "Relationshin Consequences",
    tone: "Apologetic",
    time: "Morning",
    author: "Hoffbro",
    message_contents: "Sorry I screwed your girlfriend last night bro I was wasted"
}

Yo, Hoffbro, I thought we were bros, man! What happened last night - no respect! You know I don't play with that mess, got no time for no consequences. But yeah, you've got a free hit of my warning shots and I won't slug it out, apology accepted, still with you. Unless you make me explain the principles of sports betting, no favors no more. #cardpac #mandobaba #brocode

> Finished chain.


"\nYo, Hoffbro, I thought we were bros, man! What happened last night - no respect! You know I don't play with that mess, got no time for no consequences. But yeah, you've got a free hit of my warning shots and I won't slug it out, apology accepted, still with you. Unless you make me explain the principles of sports betting, no favors no more. #cardpac #mandobaba #brocode"